In [1]:
import time
import pycurl
import urllib
import json
import requests
import numpy as np

In [2]:
API_ENDPOINT_URL = 'http://lauzhack.sqpub.ch/'
API_KEY = '98423jhHJDmM'

class SwissQuoteStream:
    def __init__(self):
        
        self.conn = None
        self.buffer = ''
        
        
    def setup_connection(self):
        if self.conn:
            self.conn.close()
            self.buffer = ''
        self.conn = pycurl.Curl()
        self.conn.setopt(pycurl.URL, API_ENDPOINT_URL + 'prices')
        self.conn.setopt(pycurl.WRITEFUNCTION, self.callback)
    
    def start(self, callback):
        self.callback = callback
        self.setup_connection()
        backoff_network_error = 0.25
        backoff_http_error = 5
        backoff_rate_limit = 60
        while True:
            self.setup_connection()
            try:
                self.conn.perform()
            except:
                print('Network error')
                time.sleep(backoff_network_error)
                backoff_network_error = min(backoff_network_error + 1, 16)
                continue
            # HTTP Error
            sc = self.conn.getinfo(pycurl.HTTP_CODE)

            # HTTP error, use exponential back off up to 320 seconds
           # print(HTTP error %s, %s' % (sc, self.conn.errstr()))
           # print 'Waiting %s seconds' % backoff_http_error
            time.sleep(backoff_http_error)
            backoff_http_error = min(backoff_http_error * 2, 320)
            
    def buy(self, btc_amount):
        #data = str.encode('BUY 0.00000001 BTC ' + API_KEY)
        data = str.encode('BUY ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data=data)
        return str(r.status_code) + ' ' + str(r.reason)

        
    def sell(self, btc_amount):
        #data = str.encode('SELL 0.00000001 BTC ' + API_KEY)
        data = str.encode('SELL ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data=data)
        return str(r.status_code) + ' ' + str(r.reason)
        


In [3]:
stream = SwissQuoteStream()

In [ ]:
def extract_price(data):
    return float(data.decode("utf-8").split(' ')[1][:-1])

WINDOW_MAX_SIZE = 20

window = []
epsilon = 0.01
ignore_count = 0
ignore_n = 5

normalizer_max = 5

def diff_normalizer(diff):
    if(diff > normalizer_max):
        diff_tmp = normalizer_max
    elif(diff < -normalizer_max):
        diff_tmp = -normalizer_max
    else:
        diff_tmp = diff
        
    normalized = (diff_tmp + normalizer_max) / (normalizer_max * 2)
    
    return normalized

def handle_new_price(data):
    global ignore_count

    new_price = extract_price(data)
    window.append(new_price)
    window_resized = window[-WINDOW_MAX_SIZE:]
    window_size = len(window_resized)
    
    weights = np.exp(np.linspace(-1., 0., window_size))
    weights /= weights.sum()
    
    a = np.convolve(window_resized, weights, mode='full')[:len(window_resized)]
    ema = a[window_size -1]
    
    diff = new_price - ema
    # diff > 0 -> price is above ema -> SELL SELL SELL!

    ignore_count = (ignore_count + 1 ) % ignore_n
    
    if(ignore_count == 0):
        big_enough = abs(diff) > epsilon
        
        if(not big_enough):
            print('Just HODL. Diff is: ' + str(diff))
            return
        
        END = 'http://lauzhack.sqpub.ch/teams/'

        r = requests.get(END)
        assets = r.json()[12]["assets"]
        cash = r.json()[12]["cash"]
        total = assets + cash
        
        current_percent_btc = assets / total #percentage of current bitcoin compared to total balance
        normalized_diff = 1 - diff_normalizer(diff) #percentage of bitcoin we want to have, compared to total balance
        percent_btc_diff = normalized_diff - current_percent_btc #percentage of cash to add
        amount_of_btc = percent_btc_diff * total # amount of bitcoin we want to add
        amount_in_btc = amount_of_btc / new_price #amount of equivalent bitcoin we want to add in cash
        
        #print(str(total) + ' ' + str(assets) + ' ' + str(cash) + ' ||| ' + str(diff) + ' ' + str(current_percent_btc) + ' ' + str(percent_btc_diff) + ' ' + str(amount_of_btc) + ' ' + str(amount_in_btc))
        
        if(amount_in_btc < 0):
            aa = stream.sell(abs(amount_in_btc))
            print('Diff: ' + str(normalized_diff) + '. SELL ' + str(abs(amount_in_btc)))
        else:
            aa = stream.buy(abs(amount_in_btc))
            print('Diff: ' + str(normalized_diff) + '. BUY ' + str(abs(amount_in_btc)))


In [ ]:
stream.start(handle_new_price)

Diff: 0.38907987345514816. SELL 5.1547879442899465
Diff: 0.639569681891544. BUY 6.6004938623914775
Diff: 0.6309832600044046. SELL 0.23279497935114957
Diff: 0.4036225389138508. SELL 5.994069623191424
Diff: 0.370369097634466. SELL 0.8741315187737962
Diff: 0.3696087315272052. SELL 0.018839712288150026
Diff: 0.42035867704598784. BUY 1.33757470214979
Diff: 0.47579487475918536. BUY 1.4637102124711796
Diff: 0.05372262371579384. SELL 11.107272346602404
Diff: 0.4138840492675172. BUY 9.476596217076484
Diff: 0.2656402254089244. SELL 3.901715443171476
Diff: 0.6002679900685507. BUY 8.809968851550739
Diff: 0.6334620153184005. BUY 0.87385464556223
Diff: 0.5856104802898244. SELL 1.2601135501849312
Diff: 0.512636345253577. SELL 1.9216549130463154
Diff: 0.5053720281935966. SELL 0.19128415011946495
Diff: 0.6992167384134064. BUY 5.109669211299725
Diff: 0.703333049945968. BUY 0.10885063149178183
Diff: 0.6407705789690681. SELL 1.652823988863029
Diff: 0.31079981050606875. SELL 8.689124018139168
Diff: 0.29561